# Step 1: Banking Data Hackathon - Initial Data Assessment
## Mohamed Alnor - Data Analyst

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')


print ("=" * 60)
print("BANKING DATA HACKATHON ")
print("Analyst: Mohamed Alnor")
print("=" * 60)

BANKING DATA HACKATHON 
Analyst: Mohamed Alnor


In [24]:
# Load the  all the datasets 
cards_df = pd.read_csv('cards_data.csv')
transactions_df = pd.read_csv('transactions_data.csv') 
users_df = pd.read_csv('users_data.csv')

print("All datasets loaded successfully!")

All datasets loaded successfully!


In [25]:
# Looking at the first rows of each dataset
print("Cards Data (first 5 rows):")
print(cards_df.head(), "\n")

print("Transactions Data (first 5 rows):")
print(transactions_df.head(), "\n")

print("Users Data (first 5 rows):")
print(users_df.head(), "\n")


Cards Data (first 5 rows):
     id  client_id  card_brand        card_type       card_number  expires  \
0  4524        825        Visa            Debit  4344676511950444  12/2022   
1  2731        825        Visa            Debit  4956965974959986  12/2020   
2  3701        825        Visa            Debit  4582313478255491  02/2024   
3    42        825        Visa           Credit  4879494103069057  08/2024   
4  4659        825  Mastercard  Debit (Prepaid)  5722874738736011  03/2009   

   cvv has_chip  num_cards_issued credit_limit acct_open_date  \
0  623      YES                 2       $24295        09/2002   
1  393      YES                 2       $21968        04/2014   
2  719      YES                 2       $46414        07/2003   
3  693       NO                 1       $12400        01/2003   
4   75      YES                 1          $28        09/2008   

   year_pin_last_changed card_on_dark_web  
0                   2008               No  
1                   2014 

In [26]:
# Check the columns information for each dataset
print("Cards Data Info:")
cards_df.info()
print("\n")

print("Transactions Data Info:")
transactions_df.info()
print("\n")

print("Users Data Info:")
users_df.info()


Cards Data Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6146 entries, 0 to 6145
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     6146 non-null   int64 
 1   client_id              6146 non-null   int64 
 2   card_brand             6146 non-null   object
 3   card_type              6146 non-null   object
 4   card_number            6146 non-null   int64 
 5   expires                6146 non-null   object
 6   cvv                    6146 non-null   int64 
 7   has_chip               6146 non-null   object
 8   num_cards_issued       6146 non-null   int64 
 9   credit_limit           6146 non-null   object
 10  acct_open_date         6146 non-null   object
 11  year_pin_last_changed  6146 non-null   int64 
 12  card_on_dark_web       6146 non-null   object
dtypes: int64(6), object(7)
memory usage: 624.3+ KB


Transactions Data Info:
<class 'pandas.core.frame.DataFra

In [27]:
# Data Cleaning & Preprocessing
 
def clean_monetary_columns(df, columns):
    """ Clean monetary columns by removing $ signs and converting to float"""
    for col in columns :
        if col in df.columns:
            # Remove $ sign and convert to numeric
            df[col] = df[col].str.replace('$', '').str.replace(',','')
            df[col] = pd.to_numeric(df[col], errors='coerce')
    return df

In [28]:
def preprocess_cards_data(cards_df):
    """Clean and preprocess cards data"""
    print("Processing Cards Data...")
    
    # Clean monetary columns
    cards_clean = clean_monetary_columns(cards_df.copy(), ['credit_limit'])
    
    # Convert dates
    cards_clean['expires'] = pd.to_datetime(cards_clean['expires'], format='%m/%Y', errors='coerce')
    cards_clean['acct_open_date'] = pd.to_datetime(cards_clean['acct_open_date'], format='%m/%Y', errors='coerce')
    
    # Create derived features
    cards_clean['card_age_years'] = (datetime.now() - cards_clean['acct_open_date']).dt.days / 365.25
    cards_clean['is_expired'] = cards_clean['expires'] < datetime.now()
    cards_clean['pin_change_recency'] = datetime.now().year - cards_clean['year_pin_last_changed']

    # Binary encoding for categorical variables
    cards_clean['has_chip_binary'] = (cards_clean['has_chip'] == 'YES').astype(int)
    cards_clean['dark_web_binary'] = (cards_clean['card_on_dark_web'] == 'Yes').astype(int)
    
    print(f"Cards Data: {len(cards_clean)} records processed")
    return cards_clean

In [29]:
def preprocess_users_data(users_df):
    """Clean and preprocess users data"""
    print("Processing Users Data...")
    
    # Clean monetary columns
    users_clean = clean_monetary_columns(users_df.copy(), 
                                       ['per_capita_income', 'yearly_income', 'total_debt'])
    
    # Create derived financial metrics
    users_clean['debt_to_income_ratio'] = users_clean['total_debt'] / users_clean['yearly_income']
    users_clean['debt_to_income_ratio'] = users_clean['debt_to_income_ratio'].replace([np.inf, -np.inf], np.nan)
    
    # Age-based segments
    users_clean['age_segment'] = pd.cut(users_clean['current_age'], 
                                      bins=[0, 25, 35, 50, 65, 100], 
                                      labels=['Gen Z', 'Millennial', 'Gen X', 'Boomer', 'Silent'])
    
    # Income segments
    users_clean['income_segment'] = pd.cut(users_clean['yearly_income'], 
                                         bins=[0, 30000, 60000, 100000, np.inf], 
                                         labels=['Low', 'Medium', 'High', 'Premium'])
    
    # Risk categories based on credit score
    users_clean['risk_category'] = pd.cut(users_clean['credit_score'], 
                                        bins=[0, 580, 670, 740, 850], 
                                        labels=['High Risk', 'Fair', 'Good', 'Excellent'])
    
    print(f"Users Data: {len(users_clean)} records processed")
    return users_clean


In [30]:
def preprocess_transactions_data(trans_df, sample_size=100000):
    """Clean and preprocess transactions data (with sampling for performance)"""
    print("Processing Transactions Data...")
    print(f" Large dataset detected ({len(trans_df):,} rows). Sampling {sample_size:,} for analysis...")
    
    # Sample for performance (we can adjust this based on system capacity)
    if len(trans_df) > sample_size:
        trans_sample = trans_df.sample(n=sample_size, random_state=42).copy()
    else:
        trans_sample = trans_df.copy()
    
    # Clean monetary columns
    trans_clean = clean_monetary_columns(trans_sample, ['amount'])
    
    # Convert date column
    trans_clean['date'] = pd.to_datetime(trans_clean['date'], errors='coerce')
    
    # Create time-based features
    trans_clean['year'] = trans_clean['date'].dt.year
    trans_clean['month'] = trans_clean['date'].dt.month
    trans_clean['day_of_week'] = trans_clean['date'].dt.dayofweek
    trans_clean['hour'] = trans_clean['date'].dt.hour
    trans_clean['is_weekend'] = trans_clean['day_of_week'].isin([5, 6]).astype(int)
    
    # Transaction type analysis
    trans_clean['transaction_type'] = np.where(trans_clean['amount'] < 0, 'Withdrawal', 'Payment')
    trans_clean['amount_abs'] = abs(trans_clean['amount'])
    
    # Chip usage binary
    trans_clean['chip_used'] = (trans_clean['use_chip'] == 'Chip Transaction').astype(int)
    
    # Error indicator
    trans_clean['has_error'] = trans_clean['errors'].notna().astype(int)
    
    print(f"Transactions Data: {len(trans_clean):,} records processed (sampled)")
    return trans_clean


In [31]:
def create_master_dataset(users_clean, cards_clean, trans_clean):
    """Merge all datasets into a comprehensive master dataset"""
    print(" Creating Master Dataset...")

    # Drop client_id from cards to avoid duplicate columns
    if 'client_id' in cards_clean.columns:
        cards_clean = cards_clean.drop(columns=['client_id'])
    
    # Start with transactions and join cards
    master = trans_clean.merge(cards_clean, left_on='card_id', right_on='id', 
                               how='left', suffixes=('_trans', '_card'))
    
    # Join with users (transactions.client_id -> users.id)
    master = master.merge(users_clean, left_on='client_id', right_on='id', 
                          how='left', suffixes=('', '_user'))
    
    print(f" Master Dataset: {len(master):,} records created")
    return master


In [32]:
def run_preprocessing_pipeline(users_df, cards_df, trans_df):
    """Execute complete data preprocessing pipeline"""
    print(" Starting Banking Data Preprocessing Pipeline...")
    print("=" * 50)
    
    # Process each dataset
    users_clean = preprocess_users_data(users_df)
    cards_clean = preprocess_cards_data(cards_df)
    trans_clean = preprocess_transactions_data(trans_df)
    
    # Create master dataset
    master_df = create_master_dataset(users_clean, cards_clean, trans_clean)
    
    print("=" * 50)
    print(" Preprocessing Complete!")
    print(f" Final Dataset Shape: {master_df.shape}")
    
    return {
        'users': users_clean,
        'cards': cards_clean,
        'transactions': trans_clean,
        'master': master_df
    }

In [33]:
# Data quality assessment function
def assess_data_quality(datasets):
    """Assess data quality across all datasets"""
    print("\n Data Quality Assessment:")
    print("=" * 40)
    
    for name, df in datasets.items():
        print(f"\n{name.upper()} Dataset:")
        print(f"  Shape: {df.shape}")
        print(f"  Missing Values: {df.isnull().sum().sum()}")
        print(f"  Duplicates: {df.duplicated().sum()}")
        
        if 'amount' in df.columns:
            print(f"  Amount Range: ${df['amount'].min():.2f} to ${df['amount'].max():.2f}")
        
        if 'credit_score' in df.columns:
            print(f"  Credit Score Range: {df['credit_score'].min()} to {df['credit_score'].max()}")


In [34]:
# Example usage:
datasets = run_preprocessing_pipeline(users_df, cards_df, transactions_df)
assess_data_quality(datasets)

 Starting Banking Data Preprocessing Pipeline...
Processing Users Data...
Users Data: 2000 records processed
Processing Cards Data...
Cards Data: 6146 records processed
Processing Transactions Data...
 Large dataset detected (13,305,915 rows). Sampling 100,000 for analysis...
Transactions Data: 100,000 records processed (sampled)
 Creating Master Dataset...
 Master Dataset: 100,000 records created
 Preprocessing Complete!
 Final Dataset Shape: (100000, 56)

 Data Quality Assessment:

USERS Dataset:
  Shape: (2000, 18)
  Missing Values: 1
  Duplicates: 0
  Credit Score Range: 480 to 850

CARDS Dataset:
  Shape: (6146, 18)
  Missing Values: 0
  Duplicates: 0

TRANSACTIONS Dataset:
  Shape: (100000, 21)
  Missing Values: 122624
  Duplicates: 0
  Amount Range: $-498.00 to $2554.70

MASTER Dataset:
  Shape: (100000, 56)
  Missing Values: 122740
  Duplicates: 0
  Amount Range: $-498.00 to $2554.70
  Credit Score Range: 488 to 850


In [35]:
def analyze_missing_values(datasets):
    """Comprehensive missing value analysis"""
    print(" Missing Values Analysis")
    print("=" * 50)
    
    for name, df in datasets.items():
        print(f"\n {name.upper()} Dataset Missing Values:")
        missing_summary = df.isnull().sum()
        missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)
        
        if len(missing_summary) > 0:
            missing_pct = (missing_summary / len(df) * 100).round(2)
            missing_df = pd.DataFrame({
                'Missing_Count': missing_summary,
                'Missing_Percentage': missing_pct
            })
            print(missing_df)
            
            # Identify patterns in missing data
            print(f"\n Critical Missing Columns (>10%):")
            critical_missing = missing_df[missing_df['Missing_Percentage'] > 10]
            if len(critical_missing) > 0:
                print(critical_missing)
            else:
                print(" No critical missing values found")
        else:
            print(" No missing values")
    
    return missing_summary

In [36]:
def handle_missing_values(datasets):
    """Smart missing value treatment based on data type and business logic"""
    print("\n Handling Missing Values...")
    print("=" * 50)
    
    cleaned_datasets = {}
    
    for name, df in datasets.items():
        df_clean = df.copy()
        print(f"\n Processing {name.upper()} dataset...")
        
        # Handle different types of missing values
        for col in df_clean.columns:
            missing_count = df_clean[col].isnull().sum()
            
            if missing_count > 0:
                missing_pct = (missing_count / len(df_clean)) * 100
                
                # Geographic/Location data
                if col in ['zip', 'merchant_city', 'merchant_state']:
                    df_clean[col] = df_clean[col].fillna('Unknown')
                    print(f"  ✓ {col}: Filled {missing_count} values with 'Unknown'")
                
                # Error columns (likely means no error occurred)
                elif 'error' in col.lower():
                    df_clean[col] = df_clean[col].fillna('No Error')
                    print(f"  ✓ {col}: Filled {missing_count} values with 'No Error'")
                
                # Numerical columns - use median for financial data
                elif df_clean[col].dtype in ['float64', 'int64']:
                    if 'income' in col.lower() or 'debt' in col.lower() or 'limit' in col.lower():
                        # Use median for financial metrics (more robust to outliers)
                        fill_value = df_clean[col].median()
                        df_clean[col] = df_clean[col].fillna(fill_value)
                        print(f"  ✓ {col}: Filled {missing_count} values with median ({fill_value:.2f})")
                    else:
                        # Use mean for other numerical data
                        fill_value = df_clean[col].mean()
                        df_clean[col] = df_clean[col].fillna(fill_value)
                        print(f"  ✓ {col}: Filled {missing_count} values with mean ({fill_value:.2f})")
                
                # Categorical columns
                elif df_clean[col].dtype == 'object':
                    mode_value = df_clean[col].mode()
                    if len(mode_value) > 0:
                        df_clean[col] = df_clean[col].fillna(mode_value.iloc[0])
                        print(f"  ✓ {col}: Filled {missing_count} values with mode ('{mode_value.iloc[0]}')")
                    else:
                        df_clean[col] = df_clean[col].fillna('Unknown')
                        print(f"  ✓ {col}: Filled {missing_count} values with 'Unknown'")
                
                # Date columns
                elif 'date' in col.lower():
                    # For dates, we might want to use forward fill or interpolation
                    df_clean[col] = df_clean[col].fillna(method='ffill')
                    print(f"  ✓ {col}: Forward filled {missing_count} date values")
        
        cleaned_datasets[name] = df_clean
        print(f"   {name} dataset cleaned successfully")
    
    return cleaned_datasets

In [37]:
def validate_data_quality(datasets):
    """Validate data after cleaning and create business rules"""
    print("\n Data Validation & Business Rules")
    print("=" * 50)
    
    validation_results = {}
    
    for name, df in datasets.items():
        print(f"\n {name.upper()} Dataset Validation:")
        issues = []
        
        # Check for remaining missing values
        remaining_missing = df.isnull().sum().sum()
        if remaining_missing > 0:
            issues.append(f"Still has {remaining_missing} missing values")
        else:
            print("   No missing values remaining")
        
        # Business rule validations
        if 'credit_score' in df.columns:
            invalid_credit = df[(df['credit_score'] < 300) | (df['credit_score'] > 850)]
            if len(invalid_credit) > 0:
                issues.append(f"Found {len(invalid_credit)} invalid credit scores")
            else:
                print("   All credit scores are valid (300-850)")
        
        if 'yearly_income' in df.columns:
            negative_income = df[df['yearly_income'] < 0]
            if len(negative_income) > 0:
                issues.append(f"Found {len(negative_income)} negative income values")
            else:
                print("   All income values are positive")
        
        if 'amount' in df.columns:
            # Check for extreme outliers (amounts > $10,000 might need investigation)
            extreme_amounts = df[abs(df['amount']) > 10000]
            if len(extreme_amounts) > 0:
                print(f"    Found {len(extreme_amounts)} transactions > $10,000 (flagged for review)")
        
        if 'current_age' in df.columns:
            invalid_age = df[(df['current_age'] < 18) | (df['current_age'] > 120)]
            if len(invalid_age) > 0:
                issues.append(f"Found {len(invalid_age)} invalid ages")
            else:
                print("   All ages are realistic (18-120)")
        
        validation_results[name] = issues
        
        if len(issues) == 0:
            print("   Dataset passed all validation checks!")
        else:
            print("    Issues found:")
            for issue in issues:
                print(f"    - {issue}")
    
    return validation_results


In [38]:
def create_data_summary(datasets):
    """Create comprehensive data summary for dashboard planning"""
    print("\n Data Summary for Dashboard Planning")
    print("=" * 50)
    
    summary = {}
    
    # Master dataset key metrics
    if 'master' in datasets:
        master_df = datasets['master']
        
        print("\n KEY METRICS FOR DASHBOARD:")
        
        # Customer metrics
        unique_customers = master_df['client_id'].nunique()
        print(f"   Unique Customers: {unique_customers:,}")
        
        # Transaction metrics
        total_transaction_volume = master_df['amount_abs'].sum()
        avg_transaction = master_df['amount_abs'].mean()
        print(f"   Total Transaction Volume: ${total_transaction_volume:,.2f}")
        print(f"   Average Transaction: ${avg_transaction:.2f}")
        
        # Risk metrics
        if 'dark_web_binary' in master_df.columns:
            dark_web_cards = master_df['dark_web_binary'].sum()
            print(f"   Cards on Dark Web: {dark_web_cards}")
        
        # Geographic spread
        unique_states = master_df['merchant_state'].nunique()
        unique_cities = master_df['merchant_city'].nunique()
        print(f"    Geographic Reach: {unique_states} states, {unique_cities} cities")
        
        # Time range
        if 'date' in master_df.columns:
            date_range = f"{master_df['date'].min().strftime('%Y-%m-%d')} to {master_df['date'].max().strftime('%Y-%m-%d')}"
            print(f"   Transaction Period: {date_range}")
        
        # Card brand distribution
        if 'card_brand' in master_df.columns:
            print(f"\n Card Brand Distribution:")
            brand_dist = master_df['card_brand'].value_counts()
            for brand, count in brand_dist.head().items():
                print(f"    {brand}: {count:,} ({count/len(master_df)*100:.1f}%)")
        
        summary['master_metrics'] = {
            'unique_customers': unique_customers,
            'total_volume': total_transaction_volume,
            'avg_transaction': avg_transaction,
            'geographic_states': unique_states,
            'geographic_cities': unique_cities
        }
    
    return summary


In [39]:
# save the clean data 
def save_clean_datasets(datasets, folder_path="cleaned_data"):
    """Save cleaned datasets to CSV files"""
    import os
    os.makedirs(folder_path, exist_ok=True)
    
    for name, df in datasets.items():
        file_path = os.path.join(folder_path, f"{name}_clean.csv")
        df.to_csv(file_path, index=False)
        print(f" Saved {name} dataset to {file_path}")


In [40]:
# Main function to run complete missing value treatment
def complete_data_preparation(datasets):
    """Complete data preparation pipeline"""
    print(" Starting Complete Data Preparation...")
    
    # Step 1: Analyze missing values
    missing_analysis = analyze_missing_values(datasets)
    
    # Step 2: Handle missing values
    cleaned_datasets = handle_missing_values(datasets)
    
    # Step 3: Validate data quality
    validation_results = validate_data_quality(cleaned_datasets)
    
    # Step 4: Create summary for dashboard planning
    summary = create_data_summary(cleaned_datasets)

    # Step 5: Save the clean data
    save_clean_datasets(cleaned_data, folder_path="cleaned_data")
    
    print("\n Data Preparation Complete!")
    print(" Ready for Dashboard Development!")
    
    return cleaned_datasets, validation_results, summary

# #####
cleaned_data, validation, summary = complete_data_preparation(datasets)

 Starting Complete Data Preparation...
 Missing Values Analysis

 USERS Dataset Missing Values:
             Missing_Count  Missing_Percentage
age_segment              1                0.05

 Critical Missing Columns (>10%):
 No critical missing values found

 CARDS Dataset Missing Values:
 No missing values

 TRANSACTIONS Dataset Missing Values:
                Missing_Count  Missing_Percentage
errors                  98457               98.46
zip                     12426               12.43
merchant_state          11741               11.74

 Critical Missing Columns (>10%):
                Missing_Count  Missing_Percentage
errors                  98457               98.46
zip                     12426               12.43
merchant_state          11741               11.74

 MASTER Dataset Missing Values:
                Missing_Count  Missing_Percentage
errors                  98457               98.46
zip                     12426               12.43
merchant_state          11741    